In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [ ]:
df = pd.read_csv("/content/retail_bakery_transactions (1).csv")
df.head()

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
2,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend


In [ ]:
transactions = df.groupby('Transaction')['Item'].apply(list).reset_index()
transactions.head()

,Transaction,Item
0,1,[Bread]
1,2,"[Scandinavian, Scandinavian]"
2,3,"[Hot chocolate, Jam, Cookies]"
3,4,[Muffin]
4,5,"[Coffee, Pastry, Bread]"


Take all the items that were bought in the same bill (same transaction), and put them together in a list.

people who buy Coffee often buy Pastry too.

In [ ]:
te = TransactionEncoder()
te_array = te.fit(transactions['Item']).transform(transactions['Item'])
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

df_encoded.head()

,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,Bowl Nic Pitt,Bread,Bread Pudding,Brioche and salami,Brownie,Cake,Caramel bites,Cherry me Dried fruit,Chicken Stew,Chicken sand,Chimichurri Oil,Chocolates,Christmas common,Coffee,Coffee granules,Coke,Cookies,Crepes,Crisps,Drinking chocolate spoons,Duck egg,Dulce de Leche,Eggs,Ella's Kitchen Pouches,Empanadas,Extra Salami or Feta,Fairy Doors,Farm House,Focaccia,Frittata,...,Lemon and coconut,Medialuna,Mighty Protein,Mineral water,Mortimer,Muesli,Muffin,My-5 Fruit Shoot,Nomad bag,Olum & polenta,Panatone,Pastry,Pick and Mix Bowls,Pintxos,Polenta,Postcard,Raspberry shortbread sandwich,Raw bars,Salad,Sandwich,Scandinavian,Scone,Siblings,Smoothies,Soup,Spanish Brunch,Spread,Tacos/Fajita,Tartine,Tea,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


We are turning each transaction into a row of True/False values - where each values shows whether a specific item was present or not in that transaction.

This step is necessary because FP-Growth works on a Boolean Matrix where each row is a transaction and each column is an item.

In [ ]:
frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.327205,(Bread)
1,0.029054,(Scandinavian)
2,0.058320,(Hot chocolate)
3,0.054411,(Cookies)
4,0.015003,(Jam)


This step finds out which items are frequently bought together by customers.

For example:

{'Coffee'} with support 0.15 means:

"15% of all customers bought Coffee."

{'Coffee', 'Bread'} with support 0.07 means:

"7% of all customers bought both Coffee and Bread together."

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.30)
rules_filtered = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rules_filtered.head()

,antecedents,consequents,support,confidence,lift
0,(Hot chocolate),(Coffee),0.029583,0.507246,1.060311
1,(Cookies),(Coffee),0.028209,0.518447,1.083723
2,(Muffin),(Coffee),0.018806,0.489011,1.022193
3,(Pastry),(Coffee),0.047544,0.552147,1.154168
4,(Pastry),(Bread),0.029160,0.338650,1.034977


If a customer buys [antecedents], then they also buy [consequents]."

It also shows how strong that rule is with:

Support (how often it happens overall)

Confidence (how often it's true given the antecedent)

Lift (how much more likely the consequent is compared to random chance)

In [ ]:
rules_sorted = rules.sort_values(by='lift', ascending=False)
rules_sorted.head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
16,(Toast),(Coffee),0.033597,0.478394,0.023666,0.704403,1.472431,1.0,0.007593,1.764582,0.332006,0.048464,0.433293,0.376936
18,(Spanish Brunch),(Coffee),0.018172,0.478394,0.010882,0.598837,1.251766,1.0,0.002189,1.300235,0.204851,0.022406,0.230908,0.310792
6,(Medialuna),(Coffee),0.061807,0.478394,0.035182,0.569231,1.189878,1.0,0.005614,1.210871,0.170091,0.069665,0.174148,0.321387
3,(Pastry),(Coffee),0.086107,0.478394,0.047544,0.552147,1.154168,1.0,0.006351,1.164682,0.146161,0.091968,0.141396,0.325764
14,(Alfajores),(Coffee),0.036344,0.478394,0.019651,0.540698,1.130235,1.0,0.002264,1.135648,0.119574,0.039693,0.119446,0.290888
8,(Juice),(Coffee),0.038563,0.478394,0.020602,0.534247,1.116750,1.0,0.002154,1.119919,0.108738,0.041507,0.107078,0.288656
13,(Sandwich),(Coffee),0.071844,0.478394,0.038246,0.532353,1.112792,1.0,0.003877,1.115384,0.109205,0.074701,0.103448,0.306150
10,(Cake),(Coffee),0.103856,0.478394,0.054728,0.526958,1.101515,1.0,0.005044,1.102664,0.102840,0.103745,0.093105,0.320679
17,(Scone),(Coffee),0.034548,0.478394,0.018067,0.522936,1.093107,1.0,0.001539,1.093366,0.088224,0.036507,0.085393,0.280350
1,(Cookies),(Coffee),0.054411,0.478394,0.028209,0.518447,1.083723,1.0,0.002179,1.083174,0.081700,0.055905,0.076787,0.288707


This line sorts all the association rules you’ve generated, and ranks them by how strong they are — using the lift value.

Step 1: How to Interpret Association Rules

Association rules are expressed in the form {A} -> {B}, meaning that if a customer buys item A, they are likely to also buy item B. These rules are evaluated using three key metrics: support, confidence, and lift. Support shows how frequently items A and B appear together in all transactions. Confidence measures the likelihood that item B is purchased when item A is purchased, indicating the rule's strength. Lift evaluates the association’s importance by comparing how likely item B is bought with item A versus at random; a lift greater than 1 suggests a strong positive relationship. Together, these metrics help identify meaningful patterns in customer buying behavior and support informed business decisions such as bundling or product recommendations.

Step 2: Effect of Changing Support and Confidence Thresholds

Lowering the Support Threshold
You will get more rules, including rare item combinations. This can be helpful for identifying niche or less frequent purchase patterns, but it may also result in rules that are not very meaningful or relevant.

Increasing the Support Threshold
This reduces the number of rules by focusing only on the most frequent item combinations. While this helps identify the most popular purchasing habits, it may overlook less common but potentially useful associations.

Lowering the Confidence Threshold
More rules will be included, even if they are not very strong. This can help surface possible associations but may lead to unreliable or weaker rules.

Increasing the Confidence Threshold
Only strong and reliable rules will be retained. This improves the quality of the rules but might result in missing moderately strong or uncommon associations.